In [4]:
from glob import glob
import os

data_root = "/home/maling/fanqiliang/data/others/prostate/MR"
mr_root = os.path.join(data_root, "imagesTr")
label_root = os.path.join(data_root, "labelsTr")

cases = glob(os.path.join(data_root, "imagesTr", "*.mhd"))
labels = [case.replace(mr_root, label_root).replace(".mhd", "_segmentation.mhd") for case in cases]
print(f"The number of cases: {len(labels)}")

The number of cases: 50


In [16]:
mr_root = "/home/maling/fanqiliang/output_data/medical_data/mr"
exist_cases = glob(os.path.join(mr_root, "*"))
print(f"The num of cases: {len(exist_cases)}")

The num of cases: 1020


In [15]:
import SimpleITK as sitk
import numpy as np
import cv2
from PIL import Image
import matplotlib.pyplot as plt

start_idx = len(exist_cases)

for case, label in zip(cases, labels):
    save_root = os.path.join(mr_root, f"{start_idx}")
    img = sitk.ReadImage(case)
    arr = sitk.GetArrayFromImage(img)
    for i, slice in enumerate(arr):
        save_file = os.path.join(save_root, "data", f"{i:0>4}.dcm")
        if not os.path.exists(os.path.dirname(save_file)):
            os.makedirs(os.path.dirname(save_file), exist_ok=True)
        sitk.WriteImage(sitk.GetImageFromArray(slice), save_file)
    
    label = sitk.ReadImage(label)
    arr = sitk.GetArrayFromImage(label)
    for i, slice in enumerate(arr):
        save_file = os.path.join(save_root, "label", f"{i:0>4}.jpg")
        if not os.path.exists(os.path.dirname(save_file)):
            os.makedirs(os.path.dirname(save_file), exist_ok=True)

        # slice = np.nan_to_num((slice - slice.min()) / (slice.max() - slice.min()) * 255)
        slice *= 255
        _arr = np.asarray(Image.fromarray(slice).convert("RGB"))
        cv2.imwrite(save_file, _arr)
    sitk.WriteImage(label, os.path.join(save_root, "seg_label.mhd"))

    start_idx += 1

RuntimeError: Exception thrown in SimpleITK ImageFileWriter_Execute: /tmp/SimpleITK-build/ITK/Modules/IO/GDCM/src/itkGDCMImageIO.cxx:1328:
itk::ERROR: itk::ERROR: GDCMImageIO(0x55943eb7df50): DICOM does not support this component type